---
jupyter: python3
toc: true
toc-depth: 3
toc-expand: true
number-sections: true
title: Pandas_01_Dataframe 생성
date: 2021-11-05 00:00
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: limyj0708/blog
format:
    html:
        page-layout: full
---

In [1]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display_html, display

# Dictionary에서 Dataframe 생성

`classmethod DataFrame.from_dict(data, orient='columns', dtype=None, columns=None)`

- data : {field : array-like} or {field : dict}
- orient : {‘columns’, ‘index’, ‘tight’}, default ‘columns’
  - 데이터의 방향. dict의 key가 컬럼이어야 하는 경우, columns를 넘긴다.
  - key가 row여야 한다면, index를 넘긴다.
  - tight라면, key가 ['index', 'columns', 'data', 'index_names', 'column_names']인 dict라고 가정하고 처리한다.
- dtype : dtype, default None
  - 데이터 프레임 구성 후, 강제로 적용할 변수 타입.
- columns : list, default None
  - orient에 index를 넘겼을 때 사용할 컬럼 라벨.
  - orient가 index가 아니면, ValueError가 반환된다.

In [2]:
data = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}
pd.DataFrame.from_dict(data)
# key가 컬럼으로 변환되었따. value로 들어간 리스트가 컬럼의 row 하나하나가 된다.

,col_1,col_2
0,3,a
1,2,b
2,1,c
3,0,d


In [3]:
dict_list = [
    { "id" : 1001001, "address" : "AABCC"}
    ,{ "id" : 2101001, "address" : "BBBDD"}
    ,{ "id" : 3201001, "address" : "백두산"}
    ,{ "id" : 4301001, "address" : "한라산"}
    ,{ "id" : 5401001, "address" : "몰디브"}
] # 같은 key들을 가진 딕셔너리들이 담긴 리스트
pd.DataFrame.from_dict(dict_list) # 이렇게 넣어도, key들이 컬럼이 되어 데이터프레임이 만들어진다.
# 실무적으로는 이 형태를 더 많이 쓰게 된다.

,id,address
0,1001001,AABCC
1,2101001,BBBDD
2,3201001,백두산
3,4301001,한라산
4,5401001,몰디브


# from Nested Dictionary

In [4]:
user_dict = {12: {'Category 1': {'att_1': 1, 'att_2': 'whatever'},
                  'Category 2': {'att_1': 23, 'att_2': 'another'}},
             15: {'Category 1': {'att_1': 10, 'att_2': 'foo'},
                  'Category 2': {'att_1': 30, 'att_2': 'bar'}}}

- dictionary comprehension으로 (인덱스 1, 인덱스 2) : (컬럼명1 : 값1, 컬럼명2 : 값2) 구조의 dictionary를 만든다.
- pandas의 multiindex는 tuple로 구성되어있어서, 이렇게 변환하고 dataframe을 만들면, multiindex를 가진 dataframe이 만들어진다.
- orient = index로 dataframa을 생성할 수 있다.

In [5]:
{(i,j): user_dict[i][j] for i in user_dict.keys() for j in user_dict[i].keys()}

{(12, 'Category 1'): {'att_1': 1, 'att_2': 'whatever'},
 (12, 'Category 2'): {'att_1': 23, 'att_2': 'another'},
 (15, 'Category 1'): {'att_1': 10, 'att_2': 'foo'},
 (15, 'Category 2'): {'att_1': 30, 'att_2': 'bar'}}

In [6]:
pd.DataFrame.from_dict({(i,j): user_dict[i][j] for i in user_dict.keys() for j in user_dict[i].keys()}, orient='index')

att_1     att_2
12 Category 1      1  whatever
   Category 2     23   another
15 Category 1     10       foo
   Category 2     30       bar

# Column만 존재하는 빈 Dataframe을 만들고, 내용 채워 넣기

In [7]:
df = pd.DataFrame(columns=['A','B','BB','C','D'])
# 컬럼들이 될 리스트를 columns parameter에 argument로 넘김
df

,A,B,BB,C,D


In [8]:
df['A'] = [1,3,1]
df

,A,B,BB,C,D
0,1,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN


In [9]:
df['B'] = [4,4,6]
df

,A,B,BB,C,D
0,1,4,NaN,NaN,NaN
1,3,4,NaN,NaN,NaN
2,1,6,NaN,NaN,NaN


In [10]:
df.loc[((df['A'] == 1) & (df['B'] == 4)), 'C'] = 444
df
# 컬럼 값 조건을 걸고 값을 변경

,A,B,BB,C,D
0,1,4,NaN,444,NaN
1,3,4,NaN,NaN,NaN
2,1,6,NaN,NaN,NaN


In [11]:
df.loc[(df['B'] == 4), 'C'] = 0
df
# 컬럼 값 조건을 걸고 값을 변경 2

,A,B,BB,C,D
0,1,4,NaN,0,NaN
1,3,4,NaN,0,NaN
2,1,6,NaN,NaN,NaN


In [12]:
sample_list = [1,2,3,4,5]
# 해당 데이터프레임 가장 아래에 리스트를 row로 넣음
df.loc[len(df)] = sample_list
# 이 방식은 좀 느린 편이며, 데이터프레임에 행을 추가해야 한다면
# 자료를 dictionary로 관리하다가 모든 데이터 추가가 다 끝나고 데이터프레임으로 변환하는 것이 빠름
df

,A,B,BB,C,D
0,1,4,NaN,0,NaN
1,3,4,NaN,0,NaN
2,1,6,NaN,NaN,NaN
3,1,2,3,4,5


In [13]:
df.to_parquet('df.parquet', engine='pyarrow', index=None)